### 문제
연속하는 자연수 두 개 이상을 곱해서 만들 수 있는 자연수를 크기 순으로 나열한 수열이 있습니다.
다음은 수열의 예시입니다.  
- 2, 6, 12, 20, 24, 30, 42, 56, 60, 72, 90, 110, 120, 132, 156, 182, ...  

예를 들어 2는 두 연속하는 자연수 1, 2를 곱해서 만들 수 있습니다.  
마찬가지로 6=2x3 또는 6=1x2x3이며, 20=4x5, 60=3x4x5와 같이 연속하는 자연수를 두 개 이상 곱해서 만들 수 있습니다.  
  
자연수 n이 매개변수로 주어질 때, 위 수열에서 n 번째 숫자를 return하도록 solution 함수를 완성해주세요.  

제한사항  
- n은 1 이상 1,000,000 이하인 자연수입니다.
- 정답이 10^12 이하인 경우만 입력으로 주어집니다.  
- - -
입출력 예  

n|result
---|---
1|2
2|6
4|9
9|60
10|72
11|90



In [1]:
def factorial(x, y):  # factorial(x,x) = x! , factorial(x,2) = x * (x-1) , factorial(x,n) = n회 곱
    result = 1
    for i in range(0, y):  # (x-0) * (x-1) * ... * (x-y+1)
        result *= x-i
    return result

def solution(n):
    global LOG  # 로그를 남기기 위한 On / Off
    MAX_RESULT = 10**12  # 정답이 10^12 이하인 경우만 입력으로 주어집니다.
    for i in range(MAX_RESULT):  # 정답의 범위에 의해 최대 몇 개의 곱까지 가능한지 결정
        if factorial(i, i) > MAX_RESULT:
            MAX_RANGE = i
            break
    n_set=set()  # 중복을 제거하기 위한 set
    holding_number = {}  # set에 넣을 다음 최솟값을 갖고 있음
    n_for_next = {}  # holding_number{}에서 다음 곱을 갖기 위한 정보 저장
    if LOG:  # Log를 남김. default:False
        log_arr = []
    
    for x in range(2, MAX_RANGE):
        n_for_next[x] = x
    for x in range(2, MAX_RANGE):
        holding_number[x] = factorial(x, x-1)  # {2:2, 3:6, ... , MAX_RANGE:MAX_RANGE*(MAX_RANGE-1)}

    while len(n_set) < n:  # n번째 값을 찾기 위해 중복이 제외된 len(set)의 크기를 n까지 반복
        min_value = min(holding_number.values())  # (dict)holding_number에서 가진 최소 value
        min_key = [key for key, value in holding_number.items() 
                   if value == min_value][0]  # 최솟값의 key를 저장
        n_set.add(min_value)  # set에 최솟값 저장
        
        if LOG:  # 로그를 위한 배열에 저장. default:False
            log_arr.append([min_value, n_for_next[min_key], min_key])
            
        new_n = n_for_next[min_key]+1  # 다음 곱을 얻기 위한 값 ex)2*1 -> 3*2 -> 4*3 .. -> n*(n-1)
        n_for_next[min_key] = new_n  # 방금 뽑았던 최솟값의 키에 다음 곱을 시작하는 n값 저장
        new_value = factorial(new_n, min_key)  # 방금 뽑은 키에 다음 곱 계산
        holding_number[min_key] = new_value  # 계산된 다음 곱을 방금 뽑은 키에 저장
        
    arr = list(n_set)  # 정렬을 위한 배열화
    arr.sort()
    if LOG:  # LOG=True일 경우 로그 배열 반환
        return log_arr
    else:
        return arr[n-1]

In [2]:
LOG=False
if (2 == solution(1) and 
        6 == solution(2) and
        12 == solution(3) and
        20 == solution(4) and
        24 == solution(5) and
        30 == solution(6) and
        42 == solution(7) and
        56 == solution(8) and
        60 == solution(9) and
        360 == solution(23) and
        380 == solution(24) and
        420 == solution(25)):
    print('All True')

All True


## Wrong Solution
2번째 항부터 발견되는 규칙  
2개의 곱 3개, 3개의 곱 1개로 반복  
ex)  
2x1= 2,  
3x2= 6, 4x3=12, 5x4=20, 4x3x2=24,  
6x5=30, 7x6=42, 8x7=56, 5x4x3=60,  
    ...  
    Ax(A-1), (A+1)xA, (A+2)x(A+1), Bx(B-1)x(B-2)  
      ...
        
이는 틀린 규칙이다.  
n=22까진 값이 맞으나 n=23부터 틀리게 된다.  
이 규칙대로 한다면 result = 20x19 = 380으로 나오지만,  
result = 6x5x4x3 = 360이다.  
이 규칙으로 구현한 wrong_solution(n)과 올바른 규칙으로 구현한 solotuin(n)을 비교한다.

In [3]:
LOG=False
def wrong_solution(n):
    arr = list(range(0, n+10))
    x, y = 2, 1
    for i in range(n+10):
        if i != 0 and i%4 == 0:
            arr[i] = x * (x+1) * (x+2)
            x += 1
        else:
            arr[i] = y * (y+1)
            y += 1
    arr = set(arr)
    arr = list(arr)
    arr.sort()
    return arr[n-1]

for i in range(1, 50):
    a = solution(i)
    b = wrong_solution(i)
    if a != b:
        print('!! Wrong Value at n : {}  ,  result: {} should be {}'.format(i, b, a))

!! Wrong Value at n : 23  ,  result: 380 should be 360
!! Wrong Value at n : 24  ,  result: 420 should be 380
!! Wrong Value at n : 25  ,  result: 462 should be 420
!! Wrong Value at n : 26  ,  result: 504 should be 462
!! Wrong Value at n : 27  ,  result: 506 should be 504
!! Wrong Value at n : 28  ,  result: 552 should be 506
!! Wrong Value at n : 29  ,  result: 600 should be 552
!! Wrong Value at n : 30  ,  result: 650 should be 600
!! Wrong Value at n : 31  ,  result: 702 should be 650
!! Wrong Value at n : 32  ,  result: 720 should be 702
!! Wrong Value at n : 33  ,  result: 756 should be 720
!! Wrong Value at n : 34  ,  result: 812 should be 756
!! Wrong Value at n : 35  ,  result: 870 should be 812
!! Wrong Value at n : 36  ,  result: 930 should be 840
!! Wrong Value at n : 37  ,  result: 990 should be 870
!! Wrong Value at n : 38  ,  result: 992 should be 930
!! Wrong Value at n : 39  ,  result: 1056 should be 990
!! Wrong Value at n : 40  ,  result: 1122 should be 992
!! Wrong

## 소요 시간 측정

In [4]:
LOG=False
TIMER = True  # On / Off
TIME_SIZE = 200  # 몇 회당 시간을 확인할 것인지?
LOG_SIZE = 2000  # 몇번째 n까지 시간을 확인할 것인지?
if TIMER:
    from time import time
    start_time = time()
    end_i = 0
    end_time_arr=[]
    for i in range(1, LOG_SIZE+1):
        x = solution(i)
        end_time = time()
        end_i = i
        if i%TIME_SIZE == 0:
            end_time_arr.append(time())
    end_time = time()
    n = 1
    for i in end_time_arr:
        print('size at {0:5d}  ~ {1:5d} elapsed {2:5.2f} seconds'
              .format(TIME_SIZE*(n-1), TIME_SIZE*n, round(i - start_time,2)))
        n += 1

size at     0  ~   200 elapsed  0.04 seconds
size at   200  ~   400 elapsed  0.15 seconds
size at   400  ~   600 elapsed  0.32 seconds
size at   600  ~   800 elapsed  0.56 seconds
size at   800  ~  1000 elapsed  0.86 seconds
size at  1000  ~  1200 elapsed  1.22 seconds
size at  1200  ~  1400 elapsed  1.65 seconds
size at  1400  ~  1600 elapsed  2.15 seconds
size at  1600  ~  1800 elapsed  2.71 seconds
size at  1800  ~  2000 elapsed  3.34 seconds


## LOG 출력하기  
  
출력 형태  
? = ? , ?개의 곱 , ?*? = ?  
  
2로 다시 반복이 시작될 때 띄어쓰기.

In [5]:
global LOG
LOG = True
REPEATED_VALUE = True  # 중복되는 값도 출력할지 선택. True=중복값 출력
LOG_SIZE = 102  # 몇번째 n까지 log를 출력할지?

if LOG:
    test_arr = solution(LOG_SIZE+1)
    same = 0
    was_not_2 = False
    rule, add = [], []
    n = 1
    for x in test_arr:
        s = str(x[1])
        for i in range(1, x[2]):
            s += '*'+str(x[1]-i)
        if same != x[0]:
            if x[2] == 2 and was_not_2:
                print('----'*13)
                rule.append(add)
                add = []
                was_not_2 = False
            elif x[2] != 2:
                was_not_2 = True
            if x[2] >=4:
                print('{0:10s}, {1:2d}개의 곱,  {2:14s} = {3:10d}'
                      .format('n = '+str(n), x[2], s, x[0]))
                n += 1
            else:
                print('{0:10s}, {1:2d}개의 곱,  {2:14s} = {3:10d}'
                      .format('n = '+str(n), x[2], s, x[0]))
                n += 1
            add.append(x[2])
        else:
            if REPEATED_VALUE:
                print('{0:10s}, {1:2d}개의 곱,  {2:14s} = {3:10d}  !중복값!'
                      .format('n = '+str(n), x[2], s, x[0]))
            add.append(x[2])
            if x[2] != 2:
                was_not_2 = True
        same = x[0]

n = 1     ,  2개의 곱,  2*1            =          2
n = 2     ,  2개의 곱,  3*2            =          6
n = 3     ,  3개의 곱,  3*2*1          =          6  !중복값!
----------------------------------------------------
n = 3     ,  2개의 곱,  4*3            =         12
n = 4     ,  2개의 곱,  5*4            =         20
n = 5     ,  3개의 곱,  4*3*2          =         24
n = 6     ,  4개의 곱,  4*3*2*1        =         24  !중복값!
----------------------------------------------------
n = 6     ,  2개의 곱,  6*5            =         30
n = 7     ,  2개의 곱,  7*6            =         42
n = 8     ,  2개의 곱,  8*7            =         56
n = 9     ,  3개의 곱,  5*4*3          =         60
----------------------------------------------------
n = 10    ,  2개의 곱,  9*8            =         72
n = 11    ,  2개의 곱,  10*9           =         90
n = 12    ,  2개의 곱,  11*10          =        110
n = 13    ,  3개의 곱,  6*5*4          =        120
n = 14    ,  4개의 곱,  5*4*3*2        =        120  !중복값!
n = 14    ,  5개의 곱,  5*4*3*2*1      